In [9]:
import os
from time import sleep
import json
import argparse
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [10]:
parser = argparse.ArgumentParser(description="test LLM planning abilities")
parser.add_argument(
        "--prompt_type",
        type=str,
        default='cos',
        help="type of prompt",
    )

parser.add_argument(
        "--setting",
        type=str,
        default='shuffle_both', # i: changed to shuffle_both
        help="setting",
    )

parser.add_argument(
        "--num",
        type=str,
        default='2',
        help="num",
    )
parser.add_argument(
        "--model_name",
        type=str,
        default='chatgpt',
        help="num",
    )

parser.add_argument(
        "--key",
        type=str,
        default='xxx',
        help="num",
    )

args_string = '--prompt_type=cos --setting=shuffle_both --num=2 --model_name=chatgpt --key='
args_list = args_string.split(' ')
args = parser.parse_args(args_list)

In [ ]:
apikey_filepath = '../.openai_key.txt'
with open(apikey_filepath, 'r') as f:
    key = f.read()

In [18]:
prompt_type = args.prompt_type
setting = args.setting
if args.setting == 'noshuffle':
   setting = 'shuffle_descpt'
prompt = open('../data/demo/{}_demo_{}_2.txt'.format(prompt_type,args.setting)).read()
#cot_prompt_1d = open('demo/davinci_cos_1d.txt').read()
#base_prompt = open('demo/baseline_demo.txt').read()
# Create instances for OpenAI and ChatOpenAI models
if args.model_name == 'text-davinci-003':
    openai_client = OpenAI(api_key=key, model_name=args.model_name)
else:
    # openai_client = ChatOpenAI(api_key=args.key, model_name="gpt-3.5-turbo")
    # model_name='gpt-3.5-turbo'
    # model_name='gpt-4o-mini'
    model_name='gpt-4o-2024-08-06'
    openai_client = ChatOpenAI(api_key=key, model_name=model_name)

In [19]:
# Load Data
examples_data_dir = '../data/brick_2D_50'
with open(os.path.join(examples_data_dir, 'data.json')) as f:
    data = json.load(f)

zeroshot_prompt = 'Lets think step by step, and provide the answer in the format of a sequence of bricks by a comma in the last sentence.'
constraint_prompt = 'You only need to return the sequence of removed bricks in order from top to bottom, without any explanation. So the answer is:'
# cos_1d_desc = ("Using the symbolic method to convert text to symbols, B//A means A is on top of B. "
#                "Then combine the relation together, then solve this question recursively and keep the last sentence "
#                "as the format of 'so we get the result as (the sequence of names of bricks in order):\n\n")

res_list = []
i = 0
step = 0

for item in data:
    step += 1
    print(step)

    if args.model_name == 'text-davinci-003':
        response = openai_client(prompt=zeroshot_prompt + '\n\n' + 'Question:' + '\n' + item['data'] + 'Answer:\n',
                                    temperature=0,
                                    max_tokens=2048,
                                    top_p=1,
                                    frequency_penalty=0,
                                    presence_penalty=0)
        res = response.content
    else:
        response = openai_client(messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": zeroshot_prompt + '\n\n' + 'Question:' + '\n' + item['data'] + '\nAnswer:\n'}
        ],
        max_tokens=2048,
        temperature=0)
        res = response.content

    dict_res = {'pred': res, 'label': item['label']}
    res_list.append(dict_res)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [20]:
output_path = os.path.join(examples_data_dir, f'results_{model_name}.json')
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, 'w') as outfile:
    json.dump(res_list, outfile)

print(f'Results saved to {output_path}')

Results saved to ../data/brick_2D_50/results_gpt-4o-2024-08-06.json
